In [1]:
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ipynb
from scipy.spatial import distance_matrix
import random

In [2]:
from ipynb.fs.full.datamanagement import user_filter
from ipynb.fs.full.evaluation_workflow import split_history
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance_matrix
pd.options.mode.chained_assignment = None  # default='warn'

## Data loading

In [3]:
# Read the data
CUR_DIR = os.path.abspath('')

DATA_DIR = Path(CUR_DIR).parent / "data"
FILE_PATH = DATA_DIR/'triplets_metadata_spotify.csv'
df = pd.read_csv(FILE_PATH, index_col = 0)

In [4]:
# Select a working database
user_listenings = df.groupby(['user', 'song_name', 'artist_name']).agg({'listening_count' : sum}).reset_index()
users_summary = user_listenings.groupby('user').agg({'listening_count' : sum, 'song_name' : len}).reset_index()
users_summary.columns = ['user', 'listening_count', 'track_count']


## Data management

### Users filtering

In [5]:
ids = user_filter(users_summary, nlist_min = 1, nlist_max = 500, ntracks_min = 5)
wdf = df.loc[df['user'].isin(ids),:]

In [6]:
print('Nb of ligns in df  :', len(df))
print('Nb of ligns in wdf :', len(wdf))

Nb of ligns in df  : 4982520
Nb of ligns in wdf : 3863861


In [6]:
wdf['track_id'] = wdf.song_name + '-' + wdf.artist_name
# wdf['track_id'] = 0

### Songs management

In [7]:
# Get the songs quantitative features

tracks_feats = df.drop(['user', 'listening_count'],axis = 1)
tracks_feats['track_id'] = df.song_name + '-' + df.artist_name
tracks_quanti_feats = tracks_feats.drop(['song_name', 'release', 'artist_name',
        'artist_familiarity', 'artist_hotttnesss', 'year', 'key',
       'shs_perf', 'shs_work', 'explicit', 'mode', 'time_signature', 'release_date', 'duration', 'tempo'], axis = 1).drop_duplicates()

In [8]:
# Normalize the songs quantitative feats
X = tracks_quanti_feats.drop(['track_id'], axis = 1)
X.index = tracks_quanti_feats.track_id

scaler = StandardScaler()
X[X.columns] = pd.DataFrame(scaler.fit_transform(X), index=X.index)
#X = scaler.transform(tracks_quanti_feats)
## preserve X as a dataframe


In [10]:
# Compute the pairwise distance matrix between the songs
D = pd.DataFrame(distance_matrix(X, X), index = X.index, columns = X.index)

## Raw K-Nearest Neighboor

### Split the data into hidden and apparent sets

In [ ]:
test1, test2 = split_history(wdf, 0.2)


In [ ]:
def random_k_reco(users_historic):